# Dog Breed Classifier Deployment

reference: https://cloud.google.com/blog/products/ai-machine-learning/ai-in-depth-serving-a-pytorch-text-classifier-on-ai-platform-serving-using-custom-online-prediction

## Prepare model and prediction files

- torch_model.py
- model_prediction.py

In [26]:
!pygmentize torch_model.py

# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Lint as: python3
"""Bottleneck ResNet v2 with GroupNorm and Weight Standardization."""

from collections import OrderedDict  # pylint: disable=g-importing-member

import torch
import torch.nn as nn
import torch.nn.functional as F


class StdConv2d(nn.Conv2d):
    def forward(self, x):
        w = self.weight
        v, m = torch.var_mean(w, dim=[1, 2, 3], keepdim=True, unbiased=False)
        w = (w - m) / torch.sqrt(v

In [27]:
!pygmentize model_prediction.py

import torch
import torchvision.models as models
import torchvision.transforms as transforms
from bit_model import ResNetV2
import numpy as np

from PIL import Image
from PIL import ImageFile

class DogBreedPrediction(object):
    def __init__(self, model):
        super(DogBreedPrediction, self).__init__()
        # VGG-16 pretrained model
        self._dog_detector = models.vgg16(pretrained=True)
        # BiT fine-tuned model
        self._dog_breed_classifier = model
        # https://pytorch.org/hub/pytorch_vision_vgg/
        # both models use the same preprocess step
        self._preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
        ])

        
    def _detect_dog(self, input_batch):
        with torch.no_grad():
            output = self._dog_de

## Build a package for prediction

In [28]:
!pygmentize setup.py

from setuptools import setup

REQUIRED_PACKAGES = ['torch', 'torchvision']

setup(
    name='dog_breed_classifier',
    version='0.1',
    include_package_data=True,
    scripts=['torch_model.py', 'model_prediction.py'],
    install_requires=REQUIRED_PACKAGES
)


In [29]:
!python setup.py sdist

running sdist
running egg_info
writing dog_breed_classifier.egg-info/PKG-INFO
writing dependency_links to dog_breed_classifier.egg-info/dependency_links.txt
writing requirements to dog_breed_classifier.egg-info/requires.txt
writing top-level names to dog_breed_classifier.egg-info/top_level.txt
reading manifest file 'dog_breed_classifier.egg-info/SOURCES.txt'
writing manifest file 'dog_breed_classifier.egg-info/SOURCES.txt'

running check


creating dog_breed_classifier-0.1
creating dog_breed_classifier-0.1/dog_breed_classifier.egg-info
copying files to dog_breed_classifier-0.1...
copying model_prediction.py -> dog_breed_classifier-0.1
copying setup.py -> dog_breed_classifier-0.1
copying torch_model.py -> dog_breed_classifier-0.1
copying dog_breed_classifier.egg-info/PKG-INFO -> dog_breed_classifier-0.1/dog_breed_classifier.egg-info
copying dog_breed_classifier.egg-info/SOURCES.txt -> dog_breed_classifier-0.1/dog_breed_classifier.egg-info
copying dog_breed_classifier.egg-info/dependency

In [30]:
BUCKET = 'dog-breed-classifier'
PACKAGES_DIR = 'packages'
PACKAGE_NAME = 'dog_breed_classifier-0.1.tar.gz'

In [53]:
MODEL_NAME = 'dog_breed_classifier'
MODEL_DIR = 'models'
VERSION_NAME = 'v1'
RUNTIME_VERSION = '1.13'
REGION = 'asia-northeast1'

Put the built package to cloud storage

In [38]:
!gsutil cp ./dist/{PACKAGE_NAME} gs://{BUCKET}/{PACKAGES_DIR}/{PACKAGE_NAME}

Copying file://./dist/dog_breed_classifier-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  3.9 KiB/  3.9 KiB]                                                
Operation completed over 1 objects/3.9 KiB.                                      


In [46]:
!gcloud ai-platform models create {MODEL_NAME} --regions {REGION}

Created ml engine model [projects/deep-learning-279406/models/dog_breed_classifier].


In [55]:
!gcloud alpha ml-engine versions create {VERSION_NAME} --model {MODEL_NAME} \
    --origin=gs://{BUCKET}/{MODEL_DIR}/ \
    --python-version=3.5 \
    --runtime-version={RUNTIME_VERSION} \
    --package-uris=gs://{BUCKET}/{PACKAGES_DIR}/{PACKAGE_NAME} \
    --machine-type=mls1-c4-m2 \
    --prediction-class=model.DogBreedPrediction

Creating version (this might take a few minutes)......failed.                  
ERROR: (gcloud.alpha.ml-engine.versions.create) Create Version failed. Bad model detected with error: Model requires more memory than allowed. Please try to decrease the model size and re-deploy. If you continue to experience errors, please contact support.
